In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
# Download NFT's data  
# More information: https://docs.nftport.xyz/docs/nftport/b3A6MjAzNDUzNTI-retrieve-all-nf-ts

import requests

url = "https://api.nftport.xyz/v0/transactions"

querystring = {"chain":"ethereum","type":"sale"}

headers = {
    "Content-Type": "application/json",
    "Authorization": os.environ.get("KEY_API")
}

# get the first page
response = requests.request("GET", url, headers=headers, params=querystring)

# print(response.json())


In [4]:
def convert_to_df(nft_data : dict) -> pd.DataFrame:
    df_data_nft = pd.DataFrame(nft_data.get('nft'), index=[nft_data.get('nft')['token_id']])
    df_data_price_datails = pd.DataFrame(nft_data.get('price_details'), index=[nft_data.get('nft')['token_id']])
    df_data = pd.merge(df_data_nft, df_data_price_datails, left_index=True, right_index=True)
    features = ['marketplace', 'transaction_date']
    d_features = {}
    for feature in features:
        d_features[feature] = nft_data.get(feature)
    df_data = pd.merge(df_data, pd.DataFrame(d_features, index=[nft_data.get('nft')['token_id']]), left_index=True, right_index=True)
    return df_data


# a function adds new values from transaction data to the dataset
def get_df_from_transaction_data(df_nfts : pd.DataFrame, response : requests.models.Response) -> pd.DataFrame:
    for transaction in response.json()['transactions']:
        df_nfts = pd.concat([df_nfts, convert_to_df(transaction)], ignore_index=True)
    return df_nfts


In [5]:
transaction_features = ['contract_type', 'contract_address', 'token_id', 'asset_type', 'price', 'price_usd', 'marketplace', 'transaction_date']
df_nfts_transaction = get_df_from_transaction_data(pd.DataFrame(columns=transaction_features), response)
df_nfts_transaction.head()

,contract_type,contract_address,token_id,asset_type,price,price_usd,marketplace,transaction_date,contract_address_x,contract_address_y
0,ERC721,NaN,489,ERC20,0.0270,49.986788,opensea,2022-08-18T02:25:43,0x17b19c70bfca098da3f2efef6e7fa3a1c42f5429,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
1,ERC1155,NaN,2,ERC20,0.0250,46.284063,opensea,2022-08-18T02:25:51,0x79f1d765540256aa93a96602c87a2948b4d06c11,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
2,ERC721,0x75d7ff62cf05a060bf0466807f96f770bb585b69,3582,ETH,0.0035,6.495831,opensea,2022-08-18T02:25:51,NaN,NaN
3,ERC721,0xbce3781ae7ca1a5e050bd9c4c77369867ebc307e,6417,ETH,1.2000,2227.142108,opensea,2022-08-18T02:25:51,NaN,NaN
4,ERC1155,0x495f947276749ce646f68ac8c248420045cb7b5e,1132116449130773017668250492870472447087887311...,ETH,0.0050,9.279759,opensea,2022-08-18T02:25:51,NaN,NaN


In [6]:
# a funtction adds new values from metadata to the dataset
def add_nfts(df_nfts: pd.DataFrame, response: requests.models.Response) -> pd.DataFrame:
    nft = response.json().get('nft')
    if nft == None:
        return df_nfts
    row = np.array([nft.get(el) for el in df_nfts.keys()])
    df_nfts = pd.concat([df_nfts, pd.DataFrame([row], columns = df_nfts.keys())], ignore_index=True)
    return df_nfts


def get_attributes(sr_attributes: pd.Series, response: requests.models.Response) -> pd.Series:
    nft = response.json().get('nft')
    if nft == None or nft.get('metadata') == None:
        return sr_attributes
    sr_attributes = pd.concat(
        [sr_attributes, 
        pd.Series(  
            [nft.get('metadata').get('attributes')], 
            name='attributes',
            index=[nft.get('token_id')])
        ])
    return sr_attributes

In [7]:
import time
features = ['token_id', 'chain', 'contract_address']
df_nfts_contract = pd.DataFrame(columns = features)
sr_attributes = pd.Series(dtype='object', name='attributes')
for irow in df_nfts_transaction.index:
    contract = df_nfts_transaction['contract_address'][irow]
    token = df_nfts_transaction['token_id'][irow]
    url = f"https://api.nftport.xyz/v0/nfts/{contract}/{token}"
    time.sleep(0.1)
    querystring = {"chain":"ethereum"}

    headers = {
        "Content-Type": "application/json",
        "Authorization": os.environ.get("KEY_API")
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    df_nfts_contract = add_nfts(df_nfts_contract, response)
    sr_attributes = get_attributes(sr_attributes, response)


In [8]:
df_nfts_contract.head()

,token_id,chain,contract_address
0,3582,ethereum,0x75d7ff62cf05a060bf0466807f96f770bb585b69
1,6417,ethereum,0xbce3781ae7ca1a5e050bd9c4c77369867ebc307e
2,1132116449130773017668250492870472447087887311...,ethereum,0x495f947276749ce646f68ac8c248420045cb7b5e
3,603,ethereum,0x63e8daf718b0dc39bf214f9b4dc9424841c2b267
4,5984,ethereum,0x2a73068880618acc4dc8b0b9db29aa33ca5c5396


In [9]:
df_nfts_transaction_1 = df_nfts_transaction.drop(['contract_address_x', 'contract_address_y'], axis=1)
df_nfts_contract_1 = df_nfts_contract.drop(['contract_address'], axis=1)
df_nfts_contract_1.head()


,token_id,chain
0,3582,ethereum
1,6417,ethereum
2,1132116449130773017668250492870472447087887311...,ethereum
3,603,ethereum
4,5984,ethereum


In [10]:
df_nfts_contract_1 = pd.merge(df_nfts_contract_1, sr_attributes, left_on=['token_id'], right_index=True)
df_final = pd.merge(df_nfts_transaction_1, df_nfts_contract_1, left_on=['token_id'], right_on=['token_id'], how='inner')
df_final.head()

,contract_type,contract_address,token_id,asset_type,price,price_usd,marketplace,transaction_date,chain,attributes
0,ERC721,0x75d7ff62cf05a060bf0466807f96f770bb585b69,3582,ETH,0.0035,6.495831,opensea,2022-08-18T02:25:51,ethereum,"[{'trait_type': 'RARITY', 'value': 'COMMON'}, ..."
1,ERC721,0xbce3781ae7ca1a5e050bd9c4c77369867ebc307e,6417,ETH,1.2000,2227.142108,opensea,2022-08-18T02:25:51,ethereum,"[{'trait_type': 'collrzes', 'value': 'kobold p..."
2,ERC1155,0x495f947276749ce646f68ac8c248420045cb7b5e,1132116449130773017668250492870472447087887311...,ETH,0.0050,9.279759,opensea,2022-08-18T02:25:51,ethereum,None
3,ERC721,0x63e8daf718b0dc39bf214f9b4dc9424841c2b267,603,ETH,0.0228,42.315700,opensea,2022-08-18T02:25:51,ethereum,"[{'trait_type': 'Background', 'value': 'Green'..."
4,ERC721,0x2a73068880618acc4dc8b0b9db29aa33ca5c5396,5984,ETH,0.0088,16.332375,opensea,2022-08-18T02:25:51,ethereum,"[{'trait_type': 'Background', 'value': 'Backgr..."


In [ ]:
# df_final.to_csv('Dataset_0.1.csv', index=False)